In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import pickle

from multiprocessing.dummy import Pool as ThreadPool

from collections import defaultdict

from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [7]:
electronic_years = range(2013,2019)
electronic_urls = []
for year in electronic_years:
    electronic_urls.append('https://www.billboard.com/charts/year-end/' + str(year) + '/hot-dance-electronic--songs')
print(electronic_urls)

['https://www.billboard.com/charts/year-end/2013/hot-dance-electronic--songs', 'https://www.billboard.com/charts/year-end/2014/hot-dance-electronic--songs', 'https://www.billboard.com/charts/year-end/2015/hot-dance-electronic--songs', 'https://www.billboard.com/charts/year-end/2016/hot-dance-electronic--songs', 'https://www.billboard.com/charts/year-end/2017/hot-dance-electronic--songs', 'https://www.billboard.com/charts/year-end/2018/hot-dance-electronic--songs']


In [8]:
rock_years = range(2009,2019)
rock_urls = []
for year in rock_years:
    rock_urls.append('https://www.billboard.com/charts/year-end/' + str(year) + '/hot-rock-songs')
print(rock_urls)

['https://www.billboard.com/charts/year-end/2009/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2010/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2011/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2012/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2013/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2014/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2015/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2016/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2017/hot-rock-songs', 'https://www.billboard.com/charts/year-end/2018/hot-rock-songs']


In [9]:
hip_hop_years = range(2002,2019)
hip_hop_urls = []
for year in hip_hop_years:
    hip_hop_urls.append('https://www.billboard.com/charts/year-end/' + str(year) + '/hot-r-and-and-b-hip-hop-songs')
print(hip_hop_urls)

['https://www.billboard.com/charts/year-end/2002/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2003/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2004/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2005/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2006/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2007/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2008/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2009/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2010/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2011/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2012/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-end/2013/hot-r-and-and-b-hip-hop-songs', 'https://www.billboard.com/charts/year-

In [10]:
def get_artist_and_songs(urls):
    artist_title = defaultdict(list)
    
    for url in urls:
        driver = webdriver.Chrome("/usr/local/bin/chromedriver")
        driver.get(url)
        billboard_page_soup = BeautifulSoup(driver.page_source)
        driver.quit()
    
        billboard_artists = billboard_page_soup.find_all('div', attrs={'class': 'ye-chart-item__artist'})
        billboard_songs = billboard_page_soup.find_all('div', attrs={'class': 'ye-chart-item__title'})
        for artist,song in zip(billboard_artists, billboard_songs):
            artist_text = artist.text.replace('\n', '')
            artist_text = artist_text.split('&')[0].strip()
            artist_text = artist_text.split(' Featuring ')[0].strip()
            artist_text = artist_text.split(' With ')[0].strip()
            artist_text = artist_text.split(' And ')[0].strip()
            artist_text = artist_text.split(' X ')[0].strip()
            artist_text = artist_text.split(' VS ')[0].strip()
            artist_text = artist_text.split(' x ')[0].strip()
            artist_text = artist_text.split(' vs ')[0].strip()
            artist_text = artist_text.split('+')[0].strip()
            artist_text = artist_text.split(',')[0]
            artist_title[artist_text].append(song.text.replace('\n', ''))
    
    return artist_title

In [11]:
artist_and_songs_electronic = get_artist_and_songs(electronic_urls)
artist_and_songs_rock = get_artist_and_songs(rock_urls)
artist_and_songs_hip_hop = get_artist_and_songs(hip_hop_urls)

In [2]:
cid ="403359461f0b4827adf4b8c3b6dd87d9" 
secret = "f4837ee0941b4a769c345cc4b7b98156" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
sp.trace=False 

In [3]:
def get_audio_features_df(artist_and_songs, genre):
    track_df = pd.DataFrame()

    for artist in artist_and_songs:
        songs = artist_and_songs[artist]
        for song in songs:
            song_parsed = song.replace("'", "")
            q = 'artist:' + artist + ' track:' + song_parsed
            q = q.encode(encoding='UTF-8',errors='strict')
            track_search = sp.search(q=q, type="track", limit=1)
            track_items = track_search['tracks']['items']
            if len(track_items) > 0:
                audio_features = sp.audio_features(track_items[0]['id'])
                audio_features[0].update( {'genre' : genre} ) 
                track_df = track_df.append(pd.DataFrame(audio_features[0], index=[0]))
            else:
                print('Cannot find track listing for ' + artist + '-' + song)
    
    return track_df

In [16]:
audio_electronic_df = get_audio_features_df(artist_and_songs_electronic, 'Electronic')
audio_electronic_df.shape

retrying ...1secs
retrying ...1secs
Cannot find track listing for PSY-Gentleman
retrying ...1secs
retrying ...1secs
retrying ...1secs
Cannot find track listing for Sebastian Ingrosso/Tommy Trash/John Martin-Reload
Cannot find track listing for Cazzette-Beam Me Up (Kill-Mode)
retrying ...1secs
Cannot find track listing for Ono-Walking On Thin Ice 2013
Cannot find track listing for Tony Moran VS Deborah Cooper-Heartbeat
Cannot find track listing for Lillywood-Prayer In C
Cannot find track listing for Lillywood-Prayer In C
Cannot find track listing for Skrillex-To U
Cannot find track listing for Nicky Romero VS Krewella-Legacy
Cannot find track listing for Yellow Claw-In My Room
Cannot find track listing for Marshmello-Keep It Mello
Cannot find track listing for Gryffin And Bipolar Sunshine-Whole Heart


(587, 19)

In [17]:
audio_rock_df = get_audio_features_df(artist_and_songs_rock, 'Rock')
audio_rock_df.shape

Cannot find track listing for Papa Roach-Still Swinging
retrying ...1secs
retrying ...1secs
Cannot find track listing for Breaking Benjamin-Give Me A Sign (Forever And Ever)
Cannot find track listing for Rise Against-I Don't Want To Be Here Anymore
retrying ...1secs
Cannot find track listing for The Dirty Heads-Lay Me Down
Cannot find track listing for The Dirty Heads-Spread Too Thin
Cannot find track listing for Sixx: A.M.-Lies Of The Beautiful People
Cannot find track listing for Sixx: A.M.-This Is Gonna Hurt
Cannot find track listing for Imagine Dragons-I'm So Sorry
retrying ...1secs
Cannot find track listing for Jamie N Commons-Jungle
retrying ...1secs
Cannot find track listing for U2-You're The Best Thing About Me
Cannot find track listing for U2-You're The Best Thing About Me
retrying ...1secs
retrying ...1secs
Cannot find track listing for Matt McAndrew-The Blower's Daughter
Cannot find track listing for Rag'n'Bone Man-Human
retrying ...1secs


(837, 19)

In [18]:
audio_hip_hop_df = get_audio_features_df(artist_and_songs_hip_hop, 'Hip-Hop')
audio_hip_hop_df.shape

Cannot find track listing for Usher-Love In This Club Part II
Cannot find track listing for Ja Rule-Down A** Chick
Cannot find track listing for Truth Hurts-Addictive
Cannot find track listing for P. Diddy-I Need A Girl (Part Two)
Cannot find track listing for P. Diddy-I Need A Girl (Part One)
Cannot find track listing for Ludacris- Move B***h
Cannot find track listing for Ludacris-Roll Out (My Business)
Cannot find track listing for Fat Joe-I Won't Tell
Cannot find track listing for Aaliyah-I Care 4 U
Cannot find track listing for Aaliyah-I Care 4 U
Cannot find track listing for Mary J. Blige-Love At 1st Sight
Cannot find track listing for Ginuwine-Last Chance
Cannot find track listing for Irv Gotti Presents The Inc.-Down 4 U
Cannot find track listing for Missy "Misdemeanor" Elliott-Work It
Cannot find track listing for Missy "Misdemeanor" Elliott-Take Away
Cannot find track listing for Missy "Misdemeanor" Elliott-Work It
Cannot find track listing for Missy "Misdemeanor" Elliott-Gossi

(1406, 19)

In [19]:
audio_electronic_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0.452,0.794,0,-5.151,1,0.0483,0.01110,0.001820,0.4160,0.2820,137.825,audio_features,01XFgRZfZI7oBagNf1Loml,spotify:track:01XFgRZfZI7oBagNf1Loml,https://api.spotify.com/v1/tracks/01XFgRZfZI7o...,https://api.spotify.com/v1/audio-analysis/01XF...,196664,4,Electronic
0,0.810,0.793,6,-9.404,0,0.0403,0.03780,0.000002,0.0720,0.8630,116.049,audio_features,69kOkLUCkxIZYexIgSG8rq,spotify:track:69kOkLUCkxIZYexIgSG8rq,https://api.spotify.com/v1/tracks/69kOkLUCkxIZ...,https://api.spotify.com/v1/audio-analysis/69kO...,369627,4,Electronic
0,0.832,0.659,10,-7.828,0,0.0570,0.08390,0.001140,0.0753,0.6740,100.163,audio_features,5CMjjywI0eZMixPeqNd75R,spotify:track:5CMjjywI0eZMixPeqNd75R,https://api.spotify.com/v1/tracks/5CMjjywI0eZM...,https://api.spotify.com/v1/audio-analysis/5CMj...,353893,4,Electronic
0,0.804,0.451,6,-8.207,1,0.0949,0.38800,0.000022,0.0924,0.1900,178.061,audio_features,36c4JohayB9qd64eidQMBi,spotify:track:36c4JohayB9qd64eidQMBi,https://api.spotify.com/v1/tracks/36c4JohayB9q...,https://api.spotify.com/v1/audio-analysis/36c4...,251293,4,Electronic
0,0.775,0.585,10,-9.516,0,0.0271,0.04220,0.619000,0.0770,0.5180,109.942,audio_features,2cGxRwrMyEAp8dEbuZaVv6,spotify:track:2cGxRwrMyEAp8dEbuZaVv6,https://api.spotify.com/v1/tracks/2cGxRwrMyEAp...,https://api.spotify.com/v1/audio-analysis/2cGx...,337560,4,Electronic
0,0.788,0.616,9,-10.671,0,0.0337,0.00286,0.549000,0.0952,0.6960,119.037,audio_features,0dEIca2nhcxDUV8C5QkPYb,spotify:track:0dEIca2nhcxDUV8C5QkPYb,https://api.spotify.com/v1/tracks/0dEIca2nhcxD...,https://api.spotify.com/v1/audio-analysis/0dEI...,275387,4,Electronic
0,0.740,0.682,9,-10.420,0,0.0504,0.06920,0.530000,0.3810,0.4100,113.201,audio_features,0oks4FnzhNp5QPTZtoet7c,spotify:track:0oks4FnzhNp5QPTZtoet7c,https://api.spotify.com/v1/tracks/0oks4FnzhNp5...,https://api.spotify.com/v1/audio-analysis/0oks...,544627,4,Electronic
0,0.400,0.756,11,-8.423,0,0.0492,0.00282,0.753000,0.0695,0.0924,123.828,audio_features,2KHRENHQzTIQ001nlP9Gdc,spotify:track:2KHRENHQzTIQ001nlP9Gdc,https://api.spotify.com/v1/tracks/2KHRENHQzTIQ...,https://api.spotify.com/v1/audio-analysis/2KHR...,383627,4,Electronic
0,0.810,0.793,6,-9.404,0,0.0403,0.03780,0.000002,0.0720,0.8630,116.049,audio_features,69kOkLUCkxIZYexIgSG8rq,spotify:track:69kOkLUCkxIZYexIgSG8rq,https://api.spotify.com/v1/tracks/69kOkLUCkxIZ...,https://api.spotify.com/v1/audio-analysis/69kO...,369627,4,Electronic
0,0.518,0.784,2,-5.659,1,0.0524,0.00346,0.001870,0.1710,0.5880,124.102,audio_features,4h8VwCb1MTGoLKueQ1WgbD,spotify:track:4h8VwCb1MTGoLKueQ1WgbD,https://api.spotify.com/v1/tracks/4h8VwCb1MTGo...,https://api.spotify.com/v1/audio-analysis/4h8V...,247427,4,Electronic


In [20]:
audio_electronic_df = audio_electronic_df.drop(['key', 'loudness', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'], axis=1)

In [21]:
audio_electronic_df = audio_electronic_df.reset_index()
audio_electronic_df = audio_electronic_df.drop(['index'], axis=1)
audio_electronic_df.head(5)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0.452,0.794,0.0483,0.0111,0.001820,0.4160,0.282,137.825,Electronic
1,0.810,0.793,0.0403,0.0378,0.000002,0.0720,0.863,116.049,Electronic
2,0.832,0.659,0.0570,0.0839,0.001140,0.0753,0.674,100.163,Electronic
3,0.804,0.451,0.0949,0.3880,0.000022,0.0924,0.190,178.061,Electronic
4,0.775,0.585,0.0271,0.0422,0.619000,0.0770,0.518,109.942,Electronic


In [22]:
audio_rock_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0.276,0.7150,0,-5.356,1,0.0432,0.005520,0.000417,0.2010,0.173,137.028,audio_features,5VGlqQANWDKJFl0MBG3sg2,spotify:track:5VGlqQANWDKJFl0MBG3sg2,https://api.spotify.com/v1/tracks/5VGlqQANWDKJ...,https://api.spotify.com/v1/audio-analysis/5VGl...,230760,4,Rock
0,0.542,0.9050,9,-5.653,1,0.0540,0.001720,0.010400,0.1360,0.374,153.398,audio_features,0ntQJM78wzOLVeCUAW7Y45,spotify:track:0ntQJM78wzOLVeCUAW7Y45,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,https://api.spotify.com/v1/audio-analysis/0ntQ...,203347,4,Rock
0,0.433,0.8870,4,-5.111,1,0.0470,0.005970,0.000029,0.1190,0.426,142.968,audio_features,40e9kU9Wd4vbedqSA7Io4n,spotify:track:40e9kU9Wd4vbedqSA7Io4n,https://api.spotify.com/v1/tracks/40e9kU9Wd4vb...,https://api.spotify.com/v1/audio-analysis/40e9...,180840,4,Rock
0,0.408,0.9250,5,-4.880,1,0.0533,0.008770,0.000454,0.1200,0.361,126.405,audio_features,60CirPlTOEWOUBus7HxV3B,spotify:track:60CirPlTOEWOUBus7HxV3B,https://api.spotify.com/v1/tracks/60CirPlTOEWO...,https://api.spotify.com/v1/audio-analysis/60Ci...,206187,4,Rock
0,0.408,0.9250,5,-4.880,1,0.0533,0.008770,0.000454,0.1200,0.361,126.405,audio_features,60CirPlTOEWOUBus7HxV3B,spotify:track:60CirPlTOEWOUBus7HxV3B,https://api.spotify.com/v1/tracks/60CirPlTOEWO...,https://api.spotify.com/v1/audio-analysis/60Ci...,206187,4,Rock
0,0.436,0.8710,9,-4.446,1,0.0384,0.003190,0.000473,0.1140,0.710,138.743,audio_features,4cQIbDqCZrHknxlDUjRHZ0,spotify:track:4cQIbDqCZrHknxlDUjRHZ0,https://api.spotify.com/v1/tracks/4cQIbDqCZrHk...,https://api.spotify.com/v1/audio-analysis/4cQI...,230227,4,Rock
0,0.436,0.8500,2,-4.860,1,0.0617,0.000861,0.740000,0.1280,0.537,152.644,audio_features,5LlsD7LdSMkGV4Iu0a2Zq0,spotify:track:5LlsD7LdSMkGV4Iu0a2Zq0,https://api.spotify.com/v1/tracks/5LlsD7LdSMkG...,https://api.spotify.com/v1/audio-analysis/5Lls...,183093,4,Rock
0,0.436,0.8500,2,-4.860,1,0.0617,0.000861,0.740000,0.1280,0.537,152.644,audio_features,5LlsD7LdSMkGV4Iu0a2Zq0,spotify:track:5LlsD7LdSMkGV4Iu0a2Zq0,https://api.spotify.com/v1/tracks/5LlsD7LdSMkG...,https://api.spotify.com/v1/audio-analysis/5Lls...,183093,4,Rock
0,0.448,0.8150,0,-5.951,0,0.0361,0.086700,0.686000,0.3160,0.217,124.808,audio_features,5cm47Nc1MWBVIkRFaXxvHY,spotify:track:5cm47Nc1MWBVIkRFaXxvHY,https://api.spotify.com/v1/tracks/5cm47Nc1MWBV...,https://api.spotify.com/v1/audio-analysis/5cm4...,234080,4,Rock
0,0.460,0.7960,0,-4.501,0,0.0333,0.001070,0.000000,0.1060,0.182,100.011,audio_features,0CwYG1UnRmOx8Q1EzElCIL,spotify:track:0CwYG1UnRmOx8Q1EzElCIL,https://api.spotify.com/v1/tracks/0CwYG1UnRmOx...,https://api.spotify.com/v1/audio-analysis/0CwY...,222067,4,Rock


In [23]:
audio_rock_df = audio_rock_df.drop(['key', 'loudness', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'], axis=1)
audio_rock_df = audio_rock_df.reset_index()
audio_rock_df = audio_rock_df.drop(['index'], axis=1)
audio_rock_df.head(5)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0.276,0.715,0.0432,0.00552,0.000417,0.201,0.173,137.028,Rock
1,0.542,0.905,0.0540,0.00172,0.010400,0.136,0.374,153.398,Rock
2,0.433,0.887,0.0470,0.00597,0.000029,0.119,0.426,142.968,Rock
3,0.408,0.925,0.0533,0.00877,0.000454,0.120,0.361,126.405,Rock
4,0.408,0.925,0.0533,0.00877,0.000454,0.120,0.361,126.405,Rock


In [25]:
audio_hip_hop_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0.477,0.728,0,-5.710,1,0.0831,0.358000,0.000000,0.1100,0.6900,89.209,audio_features,6zMUIb4uce1CzpbjR3vMdN,spotify:track:6zMUIb4uce1CzpbjR3vMdN,https://api.spotify.com/v1/tracks/6zMUIb4uce1C...,https://api.spotify.com/v1/audio-analysis/6zMU...,227387,4,Hip-Hop
0,0.712,0.805,4,-3.142,0,0.0739,0.433000,0.000000,0.1790,0.8510,94.973,audio_features,6pWzCKTrKrwbUPzY8RLCoP,spotify:track:6pWzCKTrKrwbUPzY8RLCoP,https://api.spotify.com/v1/tracks/6pWzCKTrKrwb...,https://api.spotify.com/v1/audio-analysis/6pWz...,209120,4,Hip-Hop
0,0.717,0.508,7,-6.392,0,0.0290,0.263000,0.000004,0.0620,0.6810,89.473,audio_features,6CbcdeTQKj0qTaZTt0xkdk,spotify:track:6CbcdeTQKj0qTaZTt0xkdk,https://api.spotify.com/v1/tracks/6CbcdeTQKj0q...,https://api.spotify.com/v1/audio-analysis/6Cbc...,262560,4,Hip-Hop
0,0.609,0.702,8,-3.161,0,0.0350,0.350000,0.000000,0.0727,0.5130,83.637,audio_features,1YZBMqN8kHaaRPXpQTC3kb,spotify:track:1YZBMqN8kHaaRPXpQTC3kb,https://api.spotify.com/v1/tracks/1YZBMqN8kHaa...,https://api.spotify.com/v1/audio-analysis/1YZB...,297960,4,Hip-Hop
0,0.712,0.805,4,-3.142,0,0.0739,0.433000,0.000000,0.1790,0.8510,94.973,audio_features,6pWzCKTrKrwbUPzY8RLCoP,spotify:track:6pWzCKTrKrwbUPzY8RLCoP,https://api.spotify.com/v1/tracks/6pWzCKTrKrwb...,https://api.spotify.com/v1/audio-analysis/6pWz...,209120,4,Hip-Hop
0,0.601,0.621,1,-5.662,0,0.0394,0.071000,0.000004,0.2590,0.2900,94.553,audio_features,3sckKKIx96cjPXtiqPHCTB,spotify:track:3sckKKIx96cjPXtiqPHCTB,https://api.spotify.com/v1/tracks/3sckKKIx96cj...,https://api.spotify.com/v1/audio-analysis/3sck...,186307,4,Hip-Hop
0,0.561,0.897,3,-4.011,0,0.1110,0.026300,0.000551,0.0622,0.5270,79.687,audio_features,7LFcepuZqkpghbKnslItnx,spotify:track:7LFcepuZqkpghbKnslItnx,https://api.spotify.com/v1/tracks/7LFcepuZqkpg...,https://api.spotify.com/v1/audio-analysis/7LFc...,267680,4,Hip-Hop
0,0.793,0.568,1,-4.958,1,0.0459,0.047800,0.000036,0.0304,0.8060,100.005,audio_features,5PCJldueshnwqQVjS16543,spotify:track:5PCJldueshnwqQVjS16543,https://api.spotify.com/v1/tracks/5PCJldueshnw...,https://api.spotify.com/v1/audio-analysis/5PCJ...,269400,4,Hip-Hop
0,0.816,0.500,1,-4.551,1,0.0300,0.022700,0.000002,0.0733,0.6490,124.037,audio_features,7J41dYQolQJEtj3UmKLu5r,spotify:track:7J41dYQolQJEtj3UmKLu5r,https://api.spotify.com/v1/tracks/7J41dYQolQJE...,https://api.spotify.com/v1/audio-analysis/7J41...,247840,4,Hip-Hop
0,0.894,0.791,2,-4.699,1,0.1120,0.018300,0.000000,0.0388,0.5830,105.018,audio_features,5rb9QrpfcKFHM1EUbSIurX,spotify:track:5rb9QrpfcKFHM1EUbSIurX,https://api.spotify.com/v1/tracks/5rb9QrpfcKFH...,https://api.spotify.com/v1/audio-analysis/5rb9...,250373,4,Hip-Hop


In [26]:
audio_hip_hop_df = audio_hip_hop_df.drop(['key', 'loudness', 'mode', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'], axis=1)
audio_hip_hop_df = audio_hip_hop_df.reset_index()
audio_hip_hop_df = audio_hip_hop_df.drop(['index'], axis=1)
audio_hip_hop_df.head(5)

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0.477,0.728,0.0831,0.358,0.000000,0.1100,0.690,89.209,Hip-Hop
1,0.712,0.805,0.0739,0.433,0.000000,0.1790,0.851,94.973,Hip-Hop
2,0.717,0.508,0.0290,0.263,0.000004,0.0620,0.681,89.473,Hip-Hop
3,0.609,0.702,0.0350,0.350,0.000000,0.0727,0.513,83.637,Hip-Hop
4,0.712,0.805,0.0739,0.433,0.000000,0.1790,0.851,94.973,Hip-Hop


In [27]:
audio_df = audio_electronic_df.append(audio_rock_df, ignore_index=True)
audio_df = audio_df.append(audio_hip_hop_df, ignore_index=True)
audio_df.shape

(2830, 9)

In [29]:
with open('Billboard_data.pickle', 'wb') as to_write:
    pickle.dump(audio_df, to_write)

In [31]:
audio_df.to_csv(r'Billboard_data.csv')